<a href="https://colab.research.google.com/github/luismiguelcasadodiaz/IBM_SkillsBuild_IA_325/blob/main/IA_325_py_cod_ex_25_s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recomendador de canciones inteligente

## 🧠 Contexto

Estás desarrollando un sistema para una plataforma musical que quiere ofrecer **recomendaciones automáticas** basadas en características cuantitativas de las canciones, como su energía o duración.

Utilizarás el algoritmo **K-Nearest Neighbors (KNN)** de la biblioteca scikit-learn para encontrar las canciones más similares a una canción objetivo.



## 🎯 Objetivo del ejercicio

Implementar un sistema de recomendación de canciones en Python, usando el modelo de K Vecinos Más Cercanos de scikit-learn.

El sistema debe permitir recomendar canciones similares a partir de características musicales numéricas.



## 📌 Requisitos

+ 🧩 1. Clase Song: Represente una canción.
  + Atributos:

    + title (str): título de la canción.
    + artist (str): artista o grupo musical.
    + energy (float): energía de la canción (0.4 a 1.0).
    + danceability (float): cuán bailable es la canción (0.4 a 1.0).
    + duration (int): duración en segundos (180 a 300).
    + popularity (int): nivel de popularidad (50 a 100).

  + Métodos:
    + to_vector(): que devuelva una lista con los valores [energy, danceability, duration, popularity].

    + `__str__()`: que permita imprimir la canción en formato "Song Title by Artist".



+ 🤖 2. Clase SongRecommender: Usa el algoritmo de KNN de scikit-learn:

  + Atributos
    + k :número de vecinos a considerar
  + Métodos:
    + fit(song_list) debe:
      + Convertir la lista de canciones en una matriz de características numéricas.
      + Ajustar el modelo NearestNeighbors con estas características.
    + recommend(target_song) debe:
      + Obtener los k vecinos más cercanos a la canción objetivo y Devolver la lista de canciones recomendadas (sin incluir la propia canción objetivo si aparece).



+ 🔁 3. Clase SongGenerator
  + Atributos
    + num_songs: (por defecto 30).
  + Métodos
    + generate(): genera canciones aleatorias con numpy, usando nombres como "Song1", "Song2", etc., y artistas "Artist1", "Artist2", etc.



+ 🧪 4. Clase SongRecommendationExample

  + Genere una lista de canciones con SongGenerator.
  + Defina una canción personalizada como objetivo (target_song).
  + Cree una instancia de SongRecommender, la entrene con las canciones y obtenga recomendaciones.
  + Imprima por pantalla las canciones recomendadas.



## Ejemplo de salida:
```python
example = SongRecommendationExample()
example.run()
```


## Salida esperada

🎵 Recomendaciones para 'Mi Canción':
 - Song29 by Artist4
 - Song11 by Artist1
 - Song25 by Artist5


## 💡 Recomendaciones para completar el ejercicio

+ Usa numpy para generar valores aleatorios.
+ Recuerda importar NearestNeighbors desde sklearn.neighbors.
+ Asegúrate de convertir los objetos Song a vectores antes de ajustar o predecir con el modelo.
+ No incluyas la canción objetivo entre las recomendaciones (verifica si es necesario).



## Importación de librerías

In [147]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import unittest

## Definición de la clase Song

In [148]:
class Song:
  """Representa una canción con sus características numéricas.

  Atributos:
      title (str): Título de la canción.
      artist (str): Artista de la canción.
      energy (float): Energía de la canción (0.4-1.0).
      danceability (float): Bailabilidad de la canción (0.4-1.0).
      duration (int): Duración en segundos (180-300).
      popularity (int): Popularidad de la canción (50-100).
  """
  def __init__(self, title, artist, energy, danceability, duration, popularity):
    """Inicializa una nueva instancia de Song."""
    if not isinstance(title, str) or len(title) == 0:
      raise ValueError("El título debe ser una cadena de texto no vacía.")
    if not isinstance(artist, str) or len(artist) == 0:
      raise ValueError("El artista debe ser una cadena de texto no vacía")
    if not isinstance(energy, float) or energy < 0.4 or energy > 1.0:
      raise ValueError("La energía debe estar entre 0.4 y 1.0")
    if not isinstance(danceability, float) or danceability < 0.4 or danceability > 1.0:
      raise ValueError("La bailabilidad debe estar entre 0.4 y 1.0")
    if not isinstance(duration, int) or duration < 180 or duration > 300:
      raise ValueError("La duración debe estar entre 180 y 300 segundos")
    if not isinstance(popularity, int) or popularity < 50 or popularity > 100:
      raise ValueError("La popularidad debe estar entre 50 y 100")
    self.title = title
    self.artist = artist
    self.energy = energy
    self.danceability = danceability
    self.duration = duration
    self.popularity = popularity

  def to_vector(self):
    """Devuelve un vector numérico de las características."""
    return [self.energy, self.danceability, self.duration, self.popularity]
  def __str__(self):
    """Devuelve una representación en cadena de la canción."""
    return f"Song {self.title:<10} by {self.artist}"

#### Test para la clase Song

In [149]:
class TestSong(unittest.TestCase):
    def test_song_creation_valid(self):
        # Cracción de una canción válida
        song = Song("Test Title", "Test Artist", 0.7, 0.8, 240, 75)
        self.assertEqual(song.title, "Test Title")
        self.assertEqual(song.artist, "Test Artist")
        self.assertEqual(song.energy, 0.7)
        self.assertEqual(song.danceability, 0.8)
        self.assertEqual(song.duration, 240)
        self.assertEqual(song.popularity, 75)

    def test_song_creation_invalid_title(self):
        # Probando las restricciones del nombre de la canción
        with self.assertRaises(ValueError):
            Song("", "Test Artist", 0.7, 0.8, 240, 75)
        with self.assertRaises(ValueError):
            Song(123, "Test Artist", 0.7, 0.8, 240, 75)

    def test_song_creation_invalid_artist(self):
        # Probando las restricciones del artista
        with self.assertRaises(ValueError):
            Song("Test Title", "", 0.7, 0.8, 240, 75)
        with self.assertRaises(ValueError):
            Song("Test Title", 123, 0.7, 0.8, 240, 75)

    def test_song_creation_invalid_energy(self):
        # Probando los limites de la energía y un tipo erroneos
        with self.assertRaises(ValueError):
            Song("Test Title", "Test Artist", 0.3, 0.8, 240, 75)
        with self.assertRaises(ValueError):
            Song("Test Title", "Test Artist", 1.1, 0.8, 240, 75)
        with self.assertRaises(ValueError):
            Song("Test Title", "Test Artist", "invalid", 0.8, 240, 75)

    def test_song_creation_invalid_danceability(self):
        # Probando los limites de la bailabilidad y un tipo erroneos
        with self.assertRaises(ValueError):
            Song("Test Title", "Test Artist", 0.7, 0.3, 240, 75)
        with self.assertRaises(ValueError):
            Song("Test Title", "Test Artist", 0.7, 1.1, 240, 75)
        with self.assertRaises(ValueError):
            Song("Test Title", "Test Artist", 0.7, "invalid", 240, 75)

    def test_song_creation_invalid_duration(self):
        # Probando los limites de la duración y un tipo erroneos
        with self.assertRaises(ValueError):
            Song("Test Title", "Test Artist", 0.7, 0.8, 179, 75)
        with self.assertRaises(ValueError):
            Song("Test Title", "Test Artist", 0.7, 0.8, 301, 75)
        with self.assertRaises(ValueError):
            Song("Test Title", "Test Artist", 0.7, 0.8, "invalid", 75)

    def test_song_creation_invalid_popularity(self):
        # Probando los limites de la popularidad y un tipo erroneos
        with self.assertRaises(ValueError):
            Song("Test Title", "Test Artist", 0.7, 0.8, 240, 49)
        with self.assertRaises(ValueError):
            Song("Test Title", "Test Artist", 0.7, 0.8, 240, 101)
        with self.assertRaises(ValueError):
            Song("Test Title", "Test Artist", 0.7, 0.8, 240, "invalid")

    def test_to_vector(self):
        # validación del método to_vectos devuelve una lista con los parametros
        # introducidos en la creación
        song = Song("Test Title", "Test Artist", 0.7, 0.8, 240, 75)
        self.assertEqual(song.to_vector(), [0.7, 0.8, 240, 75])

    def test_str_representation(self):
        # Prueba el método __str__()
        song = Song("Test Title", "Test Artist", 0.7, 0.8, 240, 75)
        self.assertEqual(str(song), "Song Test Title by Test Artist")

In [150]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..............
----------------------------------------------------------------------
Ran 14 tests in 0.020s

OK


## Definición de la clase SongRecommender

In [151]:
class SongRecommender:
  """Recomendador de canciones basado en el algoritmo K-Nearest Neighbors.

  Utiliza características numéricas de las canciones para encontrar canciones
  similares a una canción objetivo.

  Atributos:
      k (int): El número de vecinos (canciones más cercanas) a considerar
               para la recomendación. Por defecto es 5.
      model: Una instancia de NearestNeighbors de scikit-learn.
  """
  def __init__(self, k=5):
    """Inicializa una nueva instancia de SongRecommender.

    Args:
        k (int, optional): El número de vecinos a considerar. Por defecto es 5.
    """
    self.k = k
    self.model = KNeighborsClassifier(n_neighbors=k)
    self.song_list = []

  def fit(self, song_list):
    """Entrena el modelo NearestNeighbors con una lista de canciones.

    Convierte las canciones a vectores numéricos y ajusta el modelo.

    Args:
        song_list (list): Una lista de objetos Song.
    """
    self.song_list = song_list
    X = np.array([song.to_vector() for song in song_list])
    # Usamos loa indices como etiquetas tontas para el  KNeighborsClassifier
    y = np.arange(len(song_list))
    self.model.fit(X, y)

  def recommend(self, target_song):
    """Recomienda canciones similares a una canción objetivo.

    Encuentra las k canciones más cercanas a la canción objetivo basadas en
    sus características numéricas.

    Args:
        target_song (Song): La canción objetivo para la cual se buscan
                            recomendaciones.

    Returns:
        list: Una lista de objetos Song recomendados, excluyendo la canción
              objetivo si aparece.
    """

    canciones_recomendadas = []
    target_vector = np.array(target_song.to_vector()).reshape(1, -1)
    distances, indices = self.model.kneighbors(target_vector, n_neighbors=self.k)
    print("distancias ->", distances)
    print("indices ->",indices)
    for indice in indices[0]:
      if indice != target_song:
        canciones_recomendadas.append(self.song_list[indice])
    return canciones_recomendadas

    # The indices refer to the original position in the song_list used for fitting
    #recommended_indices = indices.flatten()

    # Necesitas acceso a la lista original de canciones para devolver objetos Song.
    # Actualmente, este método no tiene acceso a esa lista.
    # Podrías considerar almacenar la lista de canciones en el objeto SongRecommender
    # durante el método fit, o pasarla al método recommend.
    # Por ahora, devolveré los índices. Necesitarías mapear estos índices
    # de vuelta a los objetos Song originales fuera de este método.

    # NOTA: Para filtrar la canción objetivo, necesitarías comparar
    # la canción objetivo con las canciones de la lista original basándote en el índice.
    # Esto requiere acceso a la lista original de canciones.

    # Ejemplo de cómo podrías filtrar SI tuvieras acceso a la lista de canciones:
    # lista_original_canciones = # ... la lista usada en fit ...
    # canciones_recomendadas = [lista_original_canciones[i] for i in recommended_indices if lista_original_canciones[i] != target_song]
    # return canciones_recomendadas

    # Devolviendo índices por ahora, asumiendo que manejarás el mapeo fuera.


## Definición de la clase SongGenerator

In [152]:
class SongGenerator:
  """Generador de canciones aleatorias.

  Atributos:
      num_songs (int): El número de canciones a generar.

  """
  def __init__(self, num_songs=30):
    """Inicializa una nueva instancia de SongGenerator.

    Args:
        num_songs (int, optional): El número de canciones a generar. Por defecto es 30.
    """
    self.num_songs = num_songs

  def generate(self):
    lista_canciones = []
    for i in range(1, self.num_songs + 1):
      title = f"Song{i}"
      # Supongo que hay hay la mitad de artistas que de canciones
      artist = f"Artist{np.random.randint(1, (self.num_songs // 2) )}"
      energy = np.random.uniform(0.4, 1.0)
      danceability = np.random.uniform(0.4, 1.0)
      duration = np.random.randint(180, 301)
      popularity = np.random.randint(50, 101)
      song = Song(title, artist, energy, danceability, duration, popularity)
      lista_canciones.append(song)
    return lista_canciones

#### Tests para la calse SongGenerator

In [153]:
class TestSongGenerator(unittest.TestCase):
    def test_song_generator_default_num_songs(self):
        # Verifica que el número por defecto de canciones se crea
        generator = SongGenerator()
        songs = generator.generate()
        self.assertEqual(len(songs), 30)

    def test_song_generator_custom_num_songs(self):
        # Verifica que se crea el número de canciones pedidas
        generator = SongGenerator(num_songs=15)
        songs = generator.generate()
        self.assertEqual(len(songs), 15)

    def test_generated_songs_are_song_objects(self):
        # Verificar que la lista devuelve canciones
        generator = SongGenerator(num_songs=5)
        songs = generator.generate()
        for song in songs:
            self.assertIsInstance(song, Song)

    def test_generated_songs_have_valid_attributes(self):
        # Validamos que los atributos de las canciones generadas están en rango
        generator = SongGenerator(num_songs=10)
        songs = generator.generate()
        for song in songs:
            self.assertIsInstance(song.title, str)
            self.assertTrue(song.title.startswith("Song"))
            self.assertIsInstance(song.artist, str)
            self.assertTrue(song.artist.startswith("Artist"))
            self.assertIsInstance(song.energy, float)
            self.assertTrue(0.4 <= song.energy <= 1.0)
            self.assertIsInstance(song.danceability, float)
            self.assertTrue(0.4 <= song.danceability <= 1.0)
            self.assertIsInstance(song.duration, int)
            self.assertTrue(180 <= song.duration <= 300) # The randint upper bound is exclusive, so 301 means up to 300
            self.assertIsInstance(song.popularity, int)
            self.assertTrue(50 <= song.popularity <= 100)

    def test_generated_song_titles_and_artists_are_unique(self):
        # Verifica la unicidad de los títulos y los artistas
        # siguiendo la convención de nombres actual (Song1, Artist1, etc.)
        generator = SongGenerator(num_songs=50)
        max_artists = 50 // 2
        songs = generator.generate()
        titles = [song.title for song in songs]
        artists = [song.artist for song in songs]
        self.assertEqual(len(titles), len(set(titles)))
        self.assertLessEqual(len(set(artists)), max_artists)

In [154]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..............
----------------------------------------------------------------------
Ran 14 tests in 0.017s

OK


## Definición de la clase SongRecommendationExample

In [155]:
class SongRecommendationExample:
  """Ejemplo de uso del recomendador de canciones."""
  def __init__(self):
    """Inicializa una nueva instancia de SongRecommenderExample."""
    self.song_generator = SongGenerator()
    self.lista_canciones = SongGenerator().generate()
    self.song_recommender = SongRecommender()
  def run(self):
    self.song_recommender.fit(self.lista_canciones)
    cancion_objetivo = Song("Mi Canción", "Mi Artista", 0.7, 0.8, 240, 75)
    recomendaciones = self.song_recommender.recommend(cancion_objetivo)
    print("Recomendaciones para 'Mi Canción':")
    for recomendacion in recomendaciones:
      print(f"- {recomendacion}")

In [156]:
example = SongRecommendationExample()
example.run()

distancias -> [[ 1.02112468 10.00672477 12.53608806 16.4935788  16.97188527]]
indices -> [[15  6 29  1 28]]
Recomendaciones para 'Mi Canción':
- Song Song16     by Artist2
- Song Song7      by Artist14
- Song Song30     by Artist7
- Song Song2      by Artist6
- Song Song29     by Artist8


In [157]:
13 //2

6